In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from itertools import combinations


df = pd.read_csv("forestfires.csv")
df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [66]:
#necessary conversions and setting up features and log area response
from sklearn.preprocessing import LabelEncoder

df['log_area'] = np.log(df['area'] + 1)

X = df.drop(columns=['area', 'log_area'])
y = df['log_area']

# Creating a LabelEncoder object
le_month = LabelEncoder()
le_day = LabelEncoder()

# Applying the actual labelEncoder to 'month' and 'day' columns
df['month'] = le_month.fit_transform(df['month'])
df['day'] = le_day.fit_transform(df['day'])

df.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,log_area
0,7,5,7,0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0,0.0
1,7,4,9,5,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0,0.0
2,7,4,9,2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0,0.0
3,8,6,7,0,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0,0.0
4,8,6,7,3,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0,0.0


In [67]:
loo = LeaveOneOut()
errors = []

# LOOCV for Linear Regression
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    errors.append((y_pred - y_test.values[0]) ** 2)

test_mse_linear = np.mean(errors)
print(f"Test MSE (Linear Regression): {test_mse_linear:.4f}")

Test MSE (Linear Regression): 2.0748


In [68]:
import numpy as np
import statsmodels.api as sm
from itertools import combinations
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, LeaveOneOut

#First we will convert out X,Y data frame values into arrays
X = np.array(X) 
y = np.array(y)  

def best_subset_selection(X, y):
    best_adj_r2 = -np.inf #lowest possible value
    best_model = None
    best_predictors = None
    
    #iterates for every predictor within X hence X.shape[1]
    for k in range(1, X.shape[1] + 1):
        
        #for every combination of predictors with k total predictors 
        for subset in combinations(range(X.shape[1]), k):
            
            # Select subset of predictors
            X_subset = X[:, list(subset)]  
            X_subset = sm.add_constant(X_subset)  # Add intercept 
            
            # now we can fit the model on the subset of predictors 
            model = sm.OLS(y, X_subset).fit()
            adj_r2 = model.rsquared_adj
            
            # saves the model if it is the best performing model
            if adj_r2 > best_adj_r2:
                best_adj_r2 = adj_r2
                best_model = model
                best_predictors = subset
                
    return best_model, best_predictors, best_adj_r2

# just running model
best_model_b, best_predictors_b, best_adj_r2_b = best_subset_selection(X, y)

# Test MSE for Best-Subset Model using LOOCV
X_best_subset = X[:, list(best_predictors_b)]  # Select only best predictors
loo = LeaveOneOut()
scores = cross_val_score(LinearRegression(), X_best_subset, y, cv=loo, scoring='neg_mean_squared_error')
test_mse_b = -scores.mean()  # ^ we have negative mean squared error above
                                                                       
print("Best adjusted R^2:", best_adj_r2_b)
print("Test MSE for Best-Subset Model:", test_mse_b)

Best adjusted R^2: 0.01167172774058478
Test MSE for Best-Subset Model: 1.9292354993506742


In [69]:
import numpy as np
import statsmodels.api as sm
from itertools import combinations
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.linear_model import LinearRegression

X = np.array(X) 
y = np.array(y)

def forward_stepwise_selection(X, y):
    
    all_remaining = list(range(X.shape[1]))  # Here we have Indices of remaining features
    
    selected = []  #takes the features we select each step
    best_adj_r2 = -np.inf 
    
    while all_remaining: #looping through all remaining predictors 
        
        adj_r2_list = []  
        for i in all_remaining:
            model = sm.OLS(y, sm.add_constant(X[:, selected + [i]])).fit()
            adj_r2_list.append((model.rsquared_adj, i))
            
        adj_r2, best_idx = max(adj_r2_list)  # going through adj_r2 list to compare to see what did the best
        
        #taking best performing model
        if adj_r2 > best_adj_r2:
            best_adj_r2 = adj_r2
            selected.append(best_idx)
            all_remaining.remove(best_idx) # we now remove that predictor and loop over and over
        else:
            break
    return selected, best_adj_r2

# Running the model
best_predictors_c, best_adj_r2_c = forward_stepwise_selection(X, y)

# Subset X to only the selected predictors
X_forward = X[:, best_predictors_c] 

model = LinearRegression()

loo = LeaveOneOut()
scores = cross_val_score(model, X_forward, y, cv=loo, scoring='neg_mean_squared_error')
test_mse_c = -np.mean(scores) 

print("Test MSE for Forward Stepwise Selection:", test_mse_c)
print("Best adjusted R^2 :", best_adj_r2_c)

Test MSE for Forward Stepwise Selection: 1.9256249821747295
Best adjusted R^2 : 0.010765430007491084


In [71]:
def backward_stepwise_selection(X, y):
    selected = list(range(X.shape[1]))  # Starting with everything
    best_adj_r2 = -np.inf

    while selected:
        adj_r2_list = []
        
        for i in selected:
            # all predictors in the model
            
            # everything except the 'i' index 
            model = sm.OLS(y, sm.add_constant(X[:, [j for j in selected if j != i]])).fit()
            adj_r2_list.append((model.rsquared_adj, i))  # appending each adj R^2
        
        # okay so here what I am doing is taking the best adj. R^2 and which predictor we removed
        # in order to achieve that maximized adj R^2 value. Hence why we have 'remove(idx)'
        adj_r2, idx = max(adj_r2_list)  
        if adj_r2 > best_adj_r2:
            break  # in case nothing improves the model
        else:
            selected.remove(idx)  # Remove that predictor

    best_model = sm.OLS(y, sm.add_constant(X[:, selected])).fit()  # taking subset of the predictors from above
    return best_model, selected, best_model.rsquared_adj

# Running the function
best_model_d, best_predictors_d, best_adj_r2_d = backward_stepwise_selection(X, y)

# Subset X to only the selected predictors and defining our model
X_best_backward = X[:, best_predictors_d]  
model = LinearRegression()

# using LOOCV and computing the test MSE
loo = LeaveOneOut()
scores = cross_val_score(model, X_best_backward, y, cv=loo, scoring='neg_mean_squared_error')
test_mse_d = -np.mean(scores)  # Negate to get positive MSE

print("Best adjusted R^2 for Backward Stepwise Model:", best_adj_r2_d)
print("Test MSE for Backward Stepwise Model:", test_mse_d)

Best adjusted R^2 for Backward Stepwise Model: 0.0018139926929584549
Test MSE for Backward Stepwise Model: 2.074779289660788


In [43]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut, GridSearchCV

alpha_values = np.logspace(-3, 3, 100)  # RANGE OF ALPHAS which will be our penalty

#ridge model creation
ridge_model = Ridge()
loo = LeaveOneOut()

param_grid = {'alpha': alpha_values} #defining alpha grid
ridge_cv = GridSearchCV(ridge_model, param_grid, cv=loo, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

#here we can find optimal penalty and test mse
best_alpha_ridge = ridge_cv.best_params_['alpha']
ridge_test_mse = -ridge_cv.best_score_

print("RIDGE: Best alpha :", best_alpha_ridge)
print("RIDGE: Test MSE :", ridge_test_mse)

RIDGE: Best alpha : 1000.0
RIDGE: Test MSE : 1.948643063564574


In [41]:
#setting up lasso model
lasso_model = Lasso()

#LOOCV 'cv = loo' 
lasso_cv = GridSearchCV(lasso_model, param_grid, cv=loo, scoring='neg_mean_squared_error')
lasso_cv.fit(X, y)

best_alpha_lasso = lasso_cv.best_params_['alpha'] #finding penalty
lasso_test_mse = -lasso_cv.best_score_ #finding test MSE

print("LASSO: Best alpha :", best_alpha_lasso) 
print("LASSO: Test MSE :", lasso_test_mse)

LASSO: Best alpha : 2.4770763559917115
LASSO: Test MSE : 1.9307692843267776


In [72]:
# (g) Summary Table
summary = pd.DataFrame({
    'Model': ['Linear Regression', 'Best Subset','Forward Selection', 'Backward Selection', 'Ridge', 'Lasso'],
    'Test MSE': [test_mse_linear,test_mse_b, test_mse_c, test_mse_d, ridge_test_mse, lasso_test_mse],
    'Best Adjusted R^2': [None, best_adj_r2_b, best_adj_r2_c, best_adj_r2_d, None, None],
    'Penalty':[None, None, None, None,best_alpha_ridge,best_alpha_lasso]
})

summary

,Model,Test MSE,Best Adjusted R^2,Penalty
0,Linear Regression,2.074779,NaN,NaN
1,Best Subset,1.929235,0.011672,NaN
2,Forward Selection,1.925625,0.010765,NaN
3,Backward Selection,2.074779,0.001814,NaN
4,Ridge,1.948643,NaN,1000.000000
5,Lasso,1.930769,NaN,2.477076
